In [1]:
import pandas as pd
import os
import glob
import csv
import warnings

In [2]:
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
folder_path = 'typesoffiles'
pattern = '*'

In [3]:
def transform_ECommerce(filename):
    xls = pd.ExcelFile(filename)
    sheet_names = xls.sheet_names
    temp_df = None
    for i, sheet_name in enumerate(sheet_names):
        if i == 0:
            df = pd.read_excel(filename, sheet_name=sheet_name).copy()
            df['VENDOR'] = sheet_name
            df['SOURCE'] = 'ECommerce'
        else:
            temp_df = pd.read_excel(filename, sheet_name=sheet_name).copy()
            temp_df['VENDOR'] = sheet_name
            temp_df['SOURCE'] = 'ECommerce'
            df = pd.concat([df, temp_df], ignore_index=True)
    return df

In [4]:
def transform_lm_map(filename):
    df = pd.read_csv(filename, encoding='unicode_escape')
    result_df = pd.DataFrame(columns=['ITEM CODE', 'UPC CODE', 'DESCRIPTION', 'MAP', 'DEALER w/ 35% DISCOUNT',
       'VENDOR', 'SOURCE', 'MSRP', 'DEALER', 'DEALER w/ 30% DISCOUNT', 'IMAP',
       'DEALER w/ 40% DISCOUNT', 'SPECIAL ORDER'])
    result_df['ITEM CODE'] = df['Item No_']
    result_df['UPC CODE'] = None
    result_df['DESCRIPTION'] = df['Description']
    result_df['MAP'] = df['MAP Price']
    result_df['DEALER w/ 35% DISCOUNT'] = None
    result_df['VENDOR'] = df['Brand']
    result_df['SOURCE'] = 'lm_map'
    result_df['MSRP'] = None
    result_df['DEALER'] = None
    result_df['DEALER w/ 30% DISCOUNT'] = None
    result_df['IMAP'] = None
    result_df['DEALER w/ 40% DISCOUNT'] = None
    result_df['SPECIAL ORDER'] = None
    return result_df

In [5]:
def read_file():
    result_df = pd.DataFrame(columns=['ITEM CODE', 'UPC CODE', 'DESCRIPTION', 'MAP', 'DEALER w/ 35% DISCOUNT',
       'VENDOR', 'SOURCE', 'MSRP', 'DEALER', 'DEALER w/ 30% DISCOUNT', 'IMAP',
       'DEALER w/ 40% DISCOUNT', 'SPECIAL ORDER'])
    filenames = glob.glob(folder_path+'/'+pattern)
    for filename in filenames:
        print(filename)
        if 'ECommerce' in filename:
            temp_data = transform_ECommerce(filename)
        elif 'lm_map' in filename:
            temp_data = transform_lm_map(filename)
        else:
            continue
        result_df = pd.concat([result_df, temp_data.copy()], ignore_index=True)
    return result_df

In [6]:
def get_title_ECommerce(desc):
    try:
        result = desc.split(',')[0]
    except:
        result = desc
    return result

In [7]:
def get_price_ECommerce(price1, price2, price3): 
    if not pd.isna(price1):
        result = price1
    else:
        if not pd.isna(price2):
            result = price2
        else:
            if not pd.isna(price3):
                result = price3
            else:
                result = None
    return result

In [8]:
def get_cost_per_item_ECommerce(price1, price2, price3, price4):
    if not pd.isna(price1):
        result = price1
    else:
        if not pd.isna(price2):
            result = price2
        else:
            if not pd.isna(price3):
                result = price3
            else:
                if not pd.isna(price4):
                    result = price4
                else:
                    result = None
    return result

In [9]:
def generate_database(data):
    database_df = pd.DataFrame()
    database_df['Handle'] = ''
    database_df['Title'] = data['DESCRIPTION'].apply(get_title_ECommerce)
    database_df['Body (HTML)'] = data['DESCRIPTION']
    database_df['Vendor'] = data['VENDOR']
    database_df['Product Category'] = 'Parts'
    database_df['Type'] = ''
    database_df['Tags'] = ''
    database_df['Published'] = 'false'
    database_df['Option1 Name'] = ''
    database_df['Option1 Value'] = ''
    database_df['Option2 Name'] = ''
    database_df['Option2 Value'] = ''
    database_df['Option3 Name'] = ''
    database_df['Option3 Value'] = ''
    database_df['Variant SKU'] = data['ITEM CODE']
    database_df['Variant Grams'] = ''
    database_df['Variant Inventory Tracker'] = ''
    database_df['Variant Inventory Qty'] = ''
    database_df['Variant Inventory Policy'] = ''
    database_df['Variant Fulfillment Service'] = 'manual'
    database_df['Variant Price'] = data.apply(lambda x: get_price_ECommerce(x['MAP'], x['IMAP'], x['MSRP']), axis=1)
    database_df['Variant Compare At Price'] = ''
    database_df['Variant Requires Shipping'] = ''
    database_df['Variant Taxable'] = ''
    database_df['Variant Barcode'] = ''
    database_df['Img Src'] = ''
    database_df['Img Position'] = ''
    database_df['Image Alt Text'] = ''
    database_df['Gift Card'] = ''
    database_df['SEO Title'] = ''
    database_df['SEO Description'] = ''
    database_df['Google Shopping / Google Product Category'] = ''
    database_df['Google Shopping / Gender'] = ''
    database_df['Google Shopping / Age Group'] = 'adult'
    database_df['Google Shopping / MPN'] = data['UPC CODE']
    database_df['Google Shopping / AdWords Grouping'] = ''
    database_df['Google Shopping / AdWords Labels'] = ''
    database_df['Google Shopping / Condition'] = 'new'
    database_df['Google Shopping / Custom Product'] = ''
    database_df['Google Shopping / Custom Label 0'] = ''
    database_df['Google Shopping / Custom Label 1'] = ''
    database_df['Google Shopping / Custom Label 2'] = ''
    database_df['Google Shopping / Custom Label 3'] = ''
    database_df['Google Shopping / Custom Label 4'] = ''
    database_df['Variant Image'] = ''
    database_df['Variant Weight Unit'] = 'grams'
    database_df['Variant Tax Code'] = ''
    database_df['Cost per item'] = data.apply(lambda x: get_cost_per_item_ECommerce(x['DEALER w/ 35% DISCOUNT'], x['DEALER w/ 30% DISCOUNT'], x['DEALER w/ 40% DISCOUNT'], x['DEALER']), axis=1)
    database_df['Price / International'] = data.apply(lambda x: get_price_ECommerce(x['MAP'], x['IMAP'], x['MSRP']), axis=1)
    database_df['Compare At Price / International'] = ''
    database_df['Status'] = 'draft'
    database_df = database_df[~database_df['Variant Price'].isnull()]
    return database_df

In [10]:
def main():
    data = read_file()
    result = generate_database(data)
    return result

In [11]:
database = main()

typesoffiles\Dealer and Distributor Pricing (4) (1).pdf
typesoffiles\ECommerce Master Pricing-UPC File.xlsx


C:\Users\Muhammad Harits R\AppData\Local\Temp\ipykernel_23812\281051360.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, temp_data.copy()], ignore_index=True)
C:\Users\Muhammad Harits R\AppData\Local\Temp\ipykernel_23812\281051360.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, temp_data.copy()], ignore_index=True)


typesoffiles\Harvia Single _ Pallet Pricing 8.25.2023 (2).pdf
typesoffiles\Internet Template June 1 2023.xlsx
typesoffiles\Karman Dealer Price List 2023.xlsx
typesoffiles\lm_map_pricing.csv
typesoffiles\SubGravity June_2021_SG.xlsx
typesoffiles\Summit to Sea email Info.pdf
typesoffiles\Thrill seeker gear Dealer Sheet - AkrobatUSA (1).xlsx
typesoffiles\~$Internet Template June 1 2023.xlsx
typesoffiles\~$Karman Dealer Price List 2023.xlsx


In [12]:
data = read_file()

typesoffiles\Dealer and Distributor Pricing (4) (1).pdf
typesoffiles\ECommerce Master Pricing-UPC File.xlsx
typesoffiles\Harvia Single _ Pallet Pricing 8.25.2023 (2).pdf
typesoffiles\Internet Template June 1 2023.xlsx
typesoffiles\Karman Dealer Price List 2023.xlsx
typesoffiles\lm_map_pricing.csv
typesoffiles\SubGravity June_2021_SG.xlsx
typesoffiles\Summit to Sea email Info.pdf
typesoffiles\Thrill seeker gear Dealer Sheet - AkrobatUSA (1).xlsx
typesoffiles\~$Internet Template June 1 2023.xlsx
typesoffiles\~$Karman Dealer Price List 2023.xlsx


C:\Users\Muhammad Harits R\AppData\Local\Temp\ipykernel_23812\281051360.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, temp_data.copy()], ignore_index=True)
C:\Users\Muhammad Harits R\AppData\Local\Temp\ipykernel_23812\281051360.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, temp_data.copy()], ignore_index=True)


In [13]:
database

,Handle,Title,Body (HTML),Vendor,Product Category,Type,Tags,Published,Option1 Name,Option1 Value,...,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Price / International,Compare At Price / International,Status
0,NaN,BEVERAGE CENTER,"BEVERAGE CENTER, CANNOT BE SOLD IN CALIFORNIA ...",BLAZE,Parts,,,false,,,...,,,,,grams,,1072.4935,1649.99,,draft
1,NaN,BEVERAGE COOLER,"BEVERAGE COOLER, OUTDOOR RATED 5.5 CUBIC FEET,...",BLAZE,Parts,,,false,,,...,,,,,grams,,1189.4935,1829.99,,draft
2,NaN,BUILT IN KIT,"BUILT IN KIT, 21'' (ELECTRIC)",BLAZE,Parts,,,false,,,...,,,,,grams,,45.4935,69.99,,draft
3,NaN,CART,"CART, GRIDDLE 30''",BLAZE,Parts,,,false,,,...,,,,,grams,,536.8935,825.99,,draft
4,NaN,CART,"CART, KAMADO 20''",BLAZE,Parts,,,false,,,...,,,,,grams,,162.4935,249.99,,draft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25429,NaN,LARGE STAINLESS STEEL RING,LARGE STAINLESS STEEL RING,CMI,Parts,,,false,,,...,,,,,grams,,NaN,17.10,,draft
25430,NaN,LARGE ALUMINUM RING,LARGE ALUMINUM RING,CMI,Parts,,,false,,,...,,,,,grams,,NaN,13.65,,draft
25431,NaN,SMALL ALUMINUM RING,SMALL ALUMINUM RING,CMI,Parts,,,false,,,...,,,,,grams,,NaN,11.75,,draft
25432,NaN,SMALL STAINLESS STEEL RING,SMALL STAINLESS STEEL RING,CMI,Parts,,,false,,,...,,,,,grams,,NaN,12.85,,draft


In [20]:
database.to_csv('product_catalog_database.csv', index=False)

In [17]:
data[data['SOURCE'] == 'lm_map']

,ITEM CODE,UPC CODE,DESCRIPTION,MAP,DEALER w/ 35% DISCOUNT,VENDOR,SOURCE,MSRP,DEALER,DEALER w/ 30% DISCOUNT,IMAP,DEALER w/ 40% DISCOUNT,SPECIAL ORDER
1519,100002,None,NOLS WINTER CAMPING,0.00,NaN,STACKPOLE BOOKS,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
1520,100005,None,HANDBOOK OF HATCHES,0.00,NaN,STACKPOLE BOOKS,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
1521,100008,None,BACKPACKING PENNSYLVANIA,0.00,NaN,STACKPOLE BOOKS,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
1522,100010,None,FIELD GUIDE EDIBLE WILD PLANTS,0.00,NaN,STACKPOLE BOOKS,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
1523,100012,None,FIELD GUIDE MEDICINAL WILD,0.00,NaN,STACKPOLE BOOKS,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25429,999971,None,LARGE STAINLESS STEEL RING,17.10,NaN,CMI,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
25430,999972,None,LARGE ALUMINUM RING,13.65,NaN,CMI,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
25431,999997,None,SMALL ALUMINUM RING,11.75,NaN,CMI,lm_map,NaN,NaN,NaN,NaN,NaN,NaN
25432,999998,None,SMALL STAINLESS STEEL RING,12.85,NaN,CMI,lm_map,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data.columns

Index(['ITEM CODE', 'UPC CODE', 'DESCRIPTION', 'MAP', 'DEALER w/ 35% DISCOUNT',
       'VENDOR', 'SOURCE', 'MSRP', 'DEALER', 'DEALER w/ 30% DISCOUNT', 'IMAP',
       'DEALER w/ 40% DISCOUNT', 'SPECIAL ORDER'],
      dtype='object')

In [19]:
empty_product = {
    'Handle': '', 'Title': '', 'Body (HTML)': '', 'Vendor': '', 'Product Category': '', 'Type': '', 'Tags': '',
    'Published': '', 'Option1 Name': '', 'Option1 Value': '', 'Option2 Name': '', 'Option2 Value': '',
    'Option3 Name': '', 'Option3 Value': '', 'Variant SKU': '', 'Variant Grams': '',
    'Variant Inventory Tracker': '', 'Variant Inventory Qty': '', 'Variant Inventory Policy': '',
    'Variant Fulfillment Service': '', 'Variant Price': '', 'Variant Compare At Price': '',
    'Variant Requires Shipping': '', 'Variant Taxable': '', 'Variant Barcode': '', 'Image Src': '',
    'Image Position': '', 'Image Alt Text': '', 'Gift Card': '', 'SEO Title': '', 'SEO Description': '',
    'Google Shopping / Google Product Category': '', 'Google Shopping / Gender': '',
    'Google Shopping / Age Group': '', 'Google Shopping / MPN': '', 'Google Shopping / AdWords Grouping': '',
    'Google Shopping / AdWords Labels': '', 'Google Shopping / Condition': '',
    'Google Shopping / Custom Product': '', 'Google Shopping / Custom Label 0': '',
    'Google Shopping / Custom Label 1': '', 'Google Shopping / Custom Label 2': '',
    'Google Shopping / Custom Label 3': '', 'Google Shopping / Custom Label 4': '', 'Variant Image': '',
    'Variant Weight Unit': '', 'Variant Tax Code': '', 'Cost per item': '', 'Price / International': '',
    'Compare At Price / International': '', 'Status': ''}